In [22]:
import pandas as pd

posts = pd.read_csv('post_img_polarity_prob.csv.gz')
len(posts), posts.columns

(17826,
 Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'funct', 'pronoun',
        'ppron', 'i', 'we', 'you', 'shehe', 'they', 'ipron', 'article', 'verb',
        'auxverb', 'past', 'present', 'future', 'adverb', 'preps', 'conj',
        'negate', 'quant', 'number', 'swear', 'social', 'family', 'friend',
        'humans', 'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad',
        'cogmech', 'insight', 'cause', 'discrep', 'tentat', 'certain', 'inhib',
        'incl', 'excl', 'percept', 'see', 'hear', 'feel', 'bio', 'body',
        'health', 'sexual', 'ingest', 'relativ', 'motion', 'space', 'time',
        'work', 'achieve', 'leisure', 'home', 'money', 'relig', 'death',
        'assent', 'nonfl', 'filler', 'post.id', 'blog.id', 'polarity',
        'img_polarity', 'img', 'wc'],
       dtype='object'))

In [23]:
posts[['polarity','img_polarity']].head(5)

,polarity,img_polarity
0,0.0,[0.17775923 0.48895797 0.33328274]
1,0.0,[0.24666864 0.22973323 0.52359813]
2,0.0,[0.24666864 0.22973323 0.52359813]
3,0.0,[0.1324832 0.57789314 0.28962368]
4,0.0,[0.03562151 0.60498923 0.3593892 ]


In [24]:
import numpy as np

def max_polarity_idx(post):
    try:
        polarities = post['img_polarity'].replace('[','').replace(']','').split(' ')
        polarities_f = [float(i) for i in polarities]
        for i,v in enumerate(polarities_f):
            if v == np.amax(polarities_f):
                return i
    except:
        return -1

def max_polarity_val(post):
    try:
        polarities = post['img_polarity'].replace('[','').replace(']','').split(' ')
        polarities_f = [float(i) for i in polarities]
        for i,v in enumerate(polarities_f):
            if v == np.amax(polarities_f):
                return v
    except:
        return 0

posts['img_max_polarity_idx'] = posts.apply(max_polarity_idx, axis=1)
posts['img_max_polarity_val'] = posts.apply(max_polarity_val, axis=1)

In [25]:
posts.head(5)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,funct,pronoun,ppron,i,we,you,shehe,...,nonfl,filler,post.id,blog.id,polarity,img_polarity,img,wc,img_max_polarity_idx,img_max_polarity_val
0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7930337763964362012,1600009401538636752,0.0,[0.17775923 0.48895797 0.33328274],4.bp.blogspot.com/-jVYHG0R0D-k/WH_-a2UXwbI/AAA...,0,1,0.488958
1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4761201206458687372,1600009401538636752,0.0,[0.24666864 0.22973323 0.52359813],4.bp.blogspot.com/-3EoATnT89Xs/WH_9ScXDzoI/AAA...,0,2,0.523598
2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1297033737519073131,1600009401538636752,0.0,[0.24666864 0.22973323 0.52359813],2.bp.blogspot.com/-3AQ5yrUxmUc/WH__GFMFyMI/AAA...,0,2,0.523598
3,3,3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3752176386259709592,1600009401538636752,0.0,[0.1324832 0.57789314 0.28962368],3.bp.blogspot.com/-s41H9n3wkHI/WHglRcxd2yI/AAA...,0,-1,0.000000
4,4,4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,366122772010640534,1600009401538636752,0.0,[0.03562151 0.60498923 0.3593892 ],2.bp.blogspot.com/-Rz9EONHW5Ro/WIKVE-yBqiI/AAA...,0,-1,0.000000


In [30]:
posts = posts[posts['wc'] > 0]

def relative_polarity(post):
    return post['polarity'] / post['wc']

posts['relative_polarity'] = posts.apply(relative_polarity, axis=1)

In [31]:
posts_sel = posts[posts['img_max_polarity_val'] >= 0.5]
posts_sel = posts_sel[posts_sel['wc'] > 10]
len(posts)

16204

In [32]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

def polarity(post):
    if post['relative_polarity'] > 0.05:
        return 2 #positive
    elif post['relative_polarity'] < 0:
        return 0 #negative
    else:
        return 1 #neutral

posts_sel['post_polarity'] = posts_sel.apply(polarity, axis=1)

acc = accuracy_score(posts_sel['img_max_polarity_idx'].values.astype(int), posts_sel['post_polarity'].values.astype(int))
print(acc)
print(classification_report(posts_sel['img_max_polarity_idx'].values.astype(int), posts_sel['post_polarity'].values.astype(int), target_names=['negative', 'neutral', 'positive']))
confusion_matrix(posts_sel['img_max_polarity_idx'].values.astype(int), posts_sel['post_polarity'].values.astype(int))

0.514186162269786
             precision    recall  f1-score   support

   negative       0.24      0.10      0.14       396
    neutral       0.62      0.72      0.66      1211
   positive       0.28      0.31      0.30       402

avg / total       0.48      0.51      0.49      2009



array([[ 39, 281,  76],
       [103, 870, 238],
       [ 18, 260, 124]])

In [34]:
posts_sel[['post_polarity','img_max_polarity_idx']].groupby(['img_max_polarity_idx']).agg(['count'])

,post_polarity
,count
img_max_polarity_idx,
0,396
1,1211
2,402


In [35]:
posts_sel[['post_polarity','img_max_polarity_idx']].groupby(['post_polarity']).agg(['count'])

,img_max_polarity_idx
,count
post_polarity,
0,160
1,1411
2,438


In [37]:
posts_sel[['blog.id','post.id','wc', 'posemo', 'negemo','relative_polarity','img_max_polarity_idx','img_max_polarity_val']].sort_values('relative_polarity',ascending=True).head(20)

,blog.id,post.id,wc,posemo,negemo,relative_polarity,img_max_polarity_idx,img_max_polarity_val
504,1602931355655786098,6178613178871563900,12,1.0,3.0,-0.166667,1,0.812242
2971,1617373788008958099,1531809388379127544,32,0.0,5.0,-0.156250,2,0.511612
12992,1672201780300804351,1341657288516511084,124,4.0,22.0,-0.145161,1,0.500368
13912,1677974486058447130,5283968270695301267,27,0.0,3.0,-0.111111,0,0.595664
11240,1662274894626835538,9036148707770749217,64,2.0,9.0,-0.109375,1,0.720136
16838,1694527588083242169,7145688845651464251,19,0.0,2.0,-0.105263,0,0.611395
14826,1683371993089354256,5132769613483756420,32,0.0,3.0,-0.093750,0,0.768649
9986,1654959187617713308,273563037090954085,96,0.0,9.0,-0.093750,1,0.756578
11499,1663774154010543227,1836888372065399997,11,0.0,1.0,-0.090909,1,0.527239
10232,1656001354177398216,5286262351300996662,56,0.0,5.0,-0.089286,1,0.572987


## Link para o Post

https://www.blogger.com/feeds/1638142496484853169/posts/default/3397322068332989517